In [1]:
import os
import torch
import random
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms, functional

In [2]:
CHARS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'M',
         'N', 'P', 'R', 'T', 'U', 'V', 'W', 'X', 'Y']

ONE_HOT = torch.eye(len(CHARS))
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class ImageDataset(Dataset):
    def __init__(self, folder, img_list, transform=None):
        self.folder = folder
        self.im_list = img_list
        self.transform = transform

    def __len__(self):
        return len(self.im_list)

    def __getitem__(self, idx):
        label = self.im_list[idx][:4]
        path = os.path.join(self.folder, self.im_list[idx])
        im = Image.open(path)
        if im.mode != 'RGB':
            im = im.convert('RGB')
        sample = {'image': im, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [4]:
class Word2OneHot(object):
    def __call__(self, sample):
        labels = list()
        for c in sample['label']:
            idx = CHARS.index(c)
            labels.append(ONE_HOT[idx])
        sample['label'] = torch.cat(labels)
        return sample

In [5]:
class ImgToTensor(object):
    def __call__(self, sample):
        np_img = np.asarray(sample['image'])
        image = np_img.transpose((2, 0, 1))  # H x W x C  -->  C x H x W
        sample['image'] = torch.from_numpy(image).float()
        return sample


In [6]:
class Normalize(transforms.Normalize):
    def __call__(self, sample):
        tensor = sample['image']
        sample['image'] = functional.normalize(
            tensor, self.mean, self.std, self.inplace)
        return sample

In [7]:
class ToGPU(object):
    def __call__(self, sample):
        sample['image'] = sample['image'].to(DEVICE)
        sample['label'] = sample['label'].float().to(DEVICE)
        return sample

In [8]:
def load_data(batch_size=4, max_m=-1, split_rate=0.2, gpu=True):
    # list images
    wd, _ = os.path.split(os.path.abspath(__file__))
    folder = os.path.join(wd, 'data')
    imgs = [i for i in os.listdir(folder) if i.endswith('jpg')]
    if not imgs:
        raise Exception('Empty folder!')
    random.seed(1)
    random.shuffle(imgs)
    point = int(split_rate * len(imgs))
    train_imgs = imgs[point:][:max_m]
    valid_imgs = imgs[:point][:max_m]

    # initialize transform
    chains = [Word2OneHot(),
              ImgToTensor(),
              Normalize([127.5, 127.5, 127.5], [128, 128, 128])]
    if gpu:
        chains.append(ToGPU())
    transform = transforms.Compose(chains)

    # load data
    train_ds = ImageDataset(folder, train_imgs, transform=transform)
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    valid_ds = ImageDataset(folder, valid_imgs, transform=transform)
    valid_dl = DataLoader(valid_ds, batch_size=batch_size)
    return train_dl, valid_dl

In [9]:
def imshow(img):
    img = img * 128 + 127.5  # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    im = Image.fromarray(npimg.astype('uint8'))
    im.show()

In [10]:
def human_time(start, end):
    s = int(end-start)
    if s < 60:
        return '{}s'.format(s)
    m = s // 60
    s = s % 60
    if m < 59:
        return '{}m {}s'.format(m, s)
    h = m // 60
    m = m % 60
    return '{}h {}m {}s'.format(h, m, s)

In [11]:
import os
import torch
import argparse
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot
import torch.nn.functional as F
from timeit import default_timer as timer
from utils import load_data, DEVICE, human_time

In [12]:
class Net(nn.Module):
    def __init__(self, gpu=False):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 18, 5)  # 18 * 32 * 116
        self.pool1 = nn.MaxPool2d(2)  # 18 * 16 * 58
        self.conv2 = nn.Conv2d(18, 48, 5)  # 48 * 12 * 54
        self.pool2 = nn.MaxPool2d(2)  # 48 * 6 * 27
        self.drop = nn.Dropout(0.5)
        self.fc1 = nn.Linear(48 * 6 * 27, 360)
        self.fc2 = nn.Linear(360, 19 * 4)

        if gpu:
            self.to(DEVICE)
            if str(DEVICE) == 'cpu':
                self.device = 'cpu'
            else:
                self.device = torch.cuda.get_device_name(0)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 48 * 6 * 27)
        x = self.drop(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x).view(-1, 4, 19)
        x = F.softmax(x, dim=2)
        x = x.view(-1, 4 * 19)
        return x

    def save(self, name, folder='./models'):
        if not os.path.exists(folder):
            os.makedirs(folder)
        path = os.path.join(folder, name)
        torch.save(self.state_dict(), path)

    def load(self, name, folder='./models'):
        path = os.path.join(folder, name)
        map_location = 'cpu' if self.device == 'cpu' else 'gpu'
        static_dict = torch.load(path, map_location)
        self.load_state_dict(static_dict)
        self.eval()

    def graph(self):
        x = torch.rand(1, 3, 40, 150)
        y = self(x)
        return make_dot(y, params=dict(self.named_parameters()))

In [13]:
def loss_batch(model, loss_func, data, opt=None):
    xb, yb = data['image'], data['label']
    batch_size = len(xb)
    out = model(xb)
    loss = loss_func(out, yb)

    single_correct, whole_correct = 0, 0
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()
    else:  # calc accuracy
        yb = yb.view(-1, 4, 19)
        out_matrix = out.view(-1, 4, 19)
        _, ans = torch.max(yb, 2)
        _, predicted = torch.max(out_matrix, 2)
        compare = (predicted == ans)
        single_correct = compare.sum().item()
        for i in range(batch_size):
            if compare[i].sum().item() == 4:
                whole_correct += 1
        del out_matrix
    loss_item = loss.item()
    del out
    del loss
    return loss_item, single_correct, whole_correct, batch_size


In [14]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl, verbose=None):
    max_acc = 0
    patience_limit = 5
    patience = 0
    for epoch in range(epochs):
        patience += 1
        running_loss = 0.0
        total_nums = 0
        model.train()
        for i, data in enumerate(train_dl):
            loss, _, _, s = loss_batch(model, loss_func, data, opt)
            if isinstance(verbose, int):
                running_loss += loss * s
                total_nums += s
                if i % verbose == verbose - 1:
                    ave_loss = running_loss / total_nums
                    print('[Epoch {}][Batch {}] got training loss: {:.6f}'
                          .format(epoch + 1, i + 1, ave_loss))
                    total_nums = 0
                    running_loss = 0.0

        model.eval()  # validate model, working for drop out layer.
        with torch.no_grad():
            losses, single, whole, batch_size = zip(
                *[loss_batch(model, loss_func, data) for data in valid_dl]
            )
        total_size = np.sum(batch_size)
        val_loss = np.sum(np.multiply(losses, batch_size)) / total_size
        single_rate = 100 * np.sum(single) / (total_size * 4)
        whole_rate = 100 * np.sum(whole) / total_size
        if single_rate > max_acc:
            patience = 0
            max_acc = single_rate
            model.save('pretrained')

        print('After epoch {}: \n'
              '\tLoss: {:.6f}\n'
              '\tSingle Acc: {:.2f}%\n'
              '\tWhole Acc: {:.2f}%'
              .format(epoch + 1, val_loss, single_rate, whole_rate))
        if patience > patience_limit:
            print('Early stop at epoch {}'.format(epoch + 1))
            break

In [15]:
def train(use_gpu=True, epochs=40, verbose=500):
    train_dl, valid_dl = load_data(batch_size=4, split_rate=0.2, gpu=use_gpu)
    model = Net(use_gpu)
    opt = optim.Adadelta(model.parameters())
    criterion = nn.BCELoss()
    start = timer()
    fit(epochs, model, criterion, opt, train_dl, valid_dl, verbose)
    end = timer()
    t = human_time(start, end)
    print('Total training time using {}: {}'.format(model.device, t))

In [16]:
train()

KeyboardInterrupt: 